<a href="https://colab.research.google.com/github/prushton2/VexScoringProgram/blob/master/RoboticsScoreProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class definitions

## Json manager class

In [0]:
class JsonManager:
  def __init__(self, path, fType): # fType should be score, comp, or team depending on which json file is being opened
    self.path = path
    self.fType = fType
  def open():
    #opens json file and returns data
    with open(self.path, mode="r") as teamFile:
      return json.load(teamFile) 
      
  def close(data):
    #opens json file and saves data
    with open(self.path, mode="w") as teamFile:
      json.dump(data, teamFile)

  def addData()

  
    
  

# Mount Google Drive


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Structure
* Json Files
  * Competitions:
    * Holds every played match sorted by competition then time
  * Scores
    * Holds every team's score, and their score over time stored in a list
  * Teams
    * Stores every team, and every match they have played.

# Backlog

### Complete

### In progress
* Create Json manager class

### TODO

* Create competition class
* Create competition ID system
* Read and compile csvs into jsons
  * Have a json for comps and a json for teams
* Print a teams stats nicely
* Create scoring system
* Show scores over time

# Sprint Task List

#### Complete

#### In progress
* Instances define paths
* Open and close function to save the values to a variable

#### TODO
* The class abstracts setting keys and values
* Creates teams and comps
* Controls teams, scores, and comps, and scores

# Json format:
## Comps:
    {
      "CompName":{
        "Matches":[
          [
            "matchName",
            [redScore, blueScore],
            [red1, red2, blue1, blue2],
            "winner"
          ]
        ],
        "Skills": [
          [
            "TeamName",
            [autonScore, driverScore]
          ]
        ]
      }
    }
## Scores:

    {
      "teamNumber": [
        score1,
        score2,
        score3...
      ]
    }

## Teams:

    {
      "teamNumber": {
        "Matches: [
          "referenceToCompsJsonMatchIndex",
        ]
      }
    }


